Proyecto E - Visor

# Exploración y preparación de datos (Calidad de datos)

## Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

## Cargar los datos

In [3]:
landis0_path = "C:\\Users\\danig\\OneDrive\\Documentos\\GitHub\\E-Visor\\data\\etsmartmeter_JULIO_2025-07-01_a_2025-07-31_1_hora (1).xlsx"
landis1_path = "C:\\Users\\danig\\OneDrive\\Documentos\\GitHub\\E-Visor\\data\\etsmartmeter_AGOSTO_2025-08-01_a_2025-08-31_1_hora.xlsx"
landis2_path = "C:\\Users\\danig\\OneDrive\\Documentos\\GitHub\\E-Visor\\data\\etsmartmeter_SEPTIEMBR_2025-09-01_a_2025-09-30_1_hora.xlsx"
landis3_path = "C:\\Users\\danig\\OneDrive\\Documentos\\GitHub\\E-Visor\\data\\etsmartmeter_OCTUBRE_2025-10-01_a_2025-10-19_1_hora.xlsx"
df0 = pd.read_excel(landis0_path)
df1 = pd.read_excel(landis1_path)
df2 = pd.read_excel(landis2_path)
df3 = pd.read_excel(landis3_path)
df1.head()

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353433 entries, 0 to 353432
Data columns (total 43 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   entity_id                353433 non-null  object 
 1   entity_type              353433 non-null  object 
 2   time_index               353433 non-null  object 
 3   v1                       353400 non-null  float64
 4   v2                       353400 non-null  float64
 5   v3                       353400 non-null  float64
 6   i1                       353400 non-null  float64
 7   i2                       353400 non-null  float64
 8   i3                       353400 non-null  float64
 9   in                       353400 non-null  float64
 10  frequency                353400 non-null  float64
 11  v1angle                  353400 non-null  float64
 12  v2angle                  353400 non-null  float64
 13  v3angle                  353400 non-null  float64
 14  i1an

In [6]:
#Códigos de bloques identificados en el dataset
df1['entity_id'].unique()
#Validar que no haya duplicados ni bloques faltantes!

array(['SmartMeter_SM_B10_ARQ', 'SmartMeter_SM_B9_SFA1',
       'SmartMeter_SM_B9_SFA2', 'SmartMeter_SM_B8_CPA',
       'SmartMeter_SM_B8_AA', 'SmartMeter_SM_B8_LABS',
       'SmartMeter_SM_HABITAT', 'SmartMeter_SM_B7_TAC',
       'SmartMeter_SM_B7_CTIC', 'SmartMeter_SM_B4_PRIM',
       'SmartMeter_SM_B5_BACH', 'SmartMeter_SM_B12_DERE',
       'SmartMeter_SM_B18_PARQ', 'SmartMeter_SM_B17_POLI',
       'SmartMeter_SM_B15_BIBL', 'SmartMeter_SM_B3_RECT'], dtype=object)

De acuerdo con el análisis realizado en el documento "Carpetas Proyecto E-Visor" hay múltiples variables que miden potencia:
- reactivepower: potencia reactiva total (no útil, o sea, energía que va y viene para sostener campos eléctricos o magnéticos, pero no realiza trabajo útil)
- reactivepower1: potencia reactiva en la fase 1
- reactivepower2: potencia reactiva en la fase 2
- reactivepower3: potencia reactiva en la fase 3
- relativethdpower: porcentaje de distorsión en la potencia
- activepower: potencia activa total (la energía que efectivamente se convierte en trabajo útil como luz, calor o movimiento)
- activepower1: potencia activa en la fase 1
- activepower2: potencia activa en la fase 2
- activepower3: potencia activa en la fase 3

Revisar si es importante la variable activeenergyimport (consumo)!

In [7]:
df1['time_index'].nunique()

353431

In [8]:
ids = df1['time_index'].value_counts()
ids[ids > 1]

time_index
2025-05-05 23:52:59.911000-05:00    2
2025-05-06 23:59:16.624000-05:00    2
Name: count, dtype: int64

## Integración de datasets

In [9]:
import pandas as pd

dfs = [df1, df2, df3]  # todos con columnas iguales (ej. time_index, entity_id, activepower, ...)
df = pd.concat(dfs, ignore_index=True)
#quita duplicados exactos (mismo medidor y timestamp)
if {'entity_id','time_index'}.issubset(df.columns):
    #ver los posibles duplicados, se asumen como duplicados los que tienen mismo 'entity_id' y 'time_index'!
    duplicados = df[df.duplicated(subset=['entity_id','time_index'], keep=False)]
    print(f"Duplicados con mismo 'entity_id' y 'time_index': {len(duplicados)}")
    df = df.drop_duplicates(subset=['entity_id','time_index'])


Duplicados con mismo 'entity_id' y 'time_index': 0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000930 entries, 0 to 1000929
Data columns (total 43 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   entity_id                1000930 non-null  object 
 1   entity_type              1000930 non-null  object 
 2   time_index               1000930 non-null  object 
 3   v1                       1000785 non-null  float64
 4   v2                       1000785 non-null  float64
 5   v3                       1000785 non-null  float64
 6   i1                       1000785 non-null  float64
 7   i2                       1000785 non-null  float64
 8   i3                       1000785 non-null  float64
 9   in                       1000785 non-null  float64
 10  frequency                1000785 non-null  float64
 11  v1angle                  1000785 non-null  float64
 12  v2angle                  1000785 non-null  float64
 13  v3angle                  1000785 non-null 

In [11]:
#Formato de fecha y hora de la variable, y zona horaria de 'time_index'
df['time_index'] = pd.to_datetime(df['time_index'], errors='coerce', utc=True)
df['time_index'] = df['time_index'].dt.tz_convert('America/Bogota')

In [12]:
df['time_index'].head()

0   2025-04-30 19:00:06.257000-05:00
1   2025-04-30 19:00:06.283000-05:00
2   2025-04-30 19:00:06.308000-05:00
3   2025-04-30 19:00:06.326000-05:00
4   2025-04-30 19:00:06.347000-05:00
Name: time_index, dtype: datetime64[ns, America/Bogota]